In [1]:
import agentpy as ap
import random
import numpy as np
import pandas as pd
from IPython.display import Image
import networkx as nx
from agentpy.tools import make_list

# For using ggplot from r for better figures 
from pyper import *
r = R(RCMD="/Library/Frameworks/R.framework/Resources/bin/R", use_pandas=True)

Python implementation of the agent-based model from "Sex specific kinship dynamics and the fitness consequences of the social environment in killer whales". The model captures social network dynamics and the evolution of helping and harming behaviour.

In [170]:
class AnimalAgent(ap.Agent):

    def setup(self):
        """ Initialise new agent variables at agent creation """
        self.age = np.random.geometric(1/self.p.N) 
        self.ageClass = self.set_age_class()
        self.sex = random.choice(['m','f'])
        # Random starting values. np array m/f rows (index 0/1), immature/adult (0/1) cols
        self.genome = np.reshape(np.random.choice(['c','d'],4), (2,2))
        self.fitness = 0
        self.num_benefit_received = 0 # How many benefits received in this round
        self.alive = True

    def set_age_class(self):
        """ Set the age class based on a threshold """
        if self.age < self.p.m:
            self.ageClass = 'immature'
        else:
            self.ageClass = 'adult'

    def increase_age(self):
        self.age += 1
        self.set_age_class()

    def get_genome_from_parents(self, parent1_genome, parent2_genome):
        """ Uniform crossover between the two parents to return the child's genome """
        choice = np.random.randint(2, size = parent1_genome.size).reshape(parent1_genome.shape).astype(bool)
        self.genome = np.where(choice, parent1_genome, parent2_genome)

    def mutate_genome(self):
        """ Random uniform mutation of each gene """

        # Create numpy mask to select each gene to mutate with given probability
        probs = [self.p.mut_prob, 1-self.p.mut_prob]
        mask = np.random.choice([True, False], self.genome.shape, p=probs)
        # Flip the values for those chosen to mutate
        self.genome[mask] = np.where(self.genome=='d', 'c', 'd')[mask]

    def get_active_strategy(self):
        """ Returns the agent's current evolved strategy for its age, given its sex """
        # m/f rows (index 0/1), immature/adult cols (0/1)
        row = 0 if self.sex == 'm' else 1
        col = 0 if self.ageClass == 'immature' else 1 
        return self.genome[row,col]

    def social_interaction(self):
        """ The agent plays the cooperation game with each social partner """
        if self.get_active_strategy() == 'c':
            for n in self.network.neighbors(self):
                n.num_benefit_received += 1
        else:
            return

    def calculate_fitness(self):
        """ Calculate and set the agent's fitness: the mean of its payoffs """
        num_neighbors = sum(1 for _ in self.network.neighbors(self))
        self.fitness = self.p.fitness_baseline  + ((self.p.benefit * self.num_benefit_received) / num_neighbors)
        if self.get_active_strategy() == 'c':
            self.fitness -= self.p.cost
        self.num_benefit_received = 0


In [195]:
class MainModel(ap.Model):

    def setup(self):
        """ Initialise the model and setup the agent population """

        # Create agent population
        self.pop = ap.AgentDList(self, self.p.N, AnimalAgent)

        # Create a random undirected network with desired mean degree
        max_edges = (self.p.N * (self.p.N - 1) / 2)
        desired_edges = (self.p.N * self.p.d) / 2
        prob_edge = desired_edges / max_edges
        graph = nx.erdos_renyi_graph(n=self.p.N, p=prob_edge)
        # Define the network in the model and give agents reference to it
        self.network = self.pop.network = ap.Network(self, graph)
        self.network.add_agents(self.pop, self.network.nodes)
    
    def kill_agent(self, agent_to_kill):
        """ Receives an agent. Removes the agent and node from the network and deletes agent """

        # Get the node the agent is on and remove the node
        agent_to_kill_node = self.network.positions[agent_to_kill]
        self.network.remove_node(agent_to_kill_node)
        # Remove agent from population
        self.pop.remove(agent_to_kill)

    def birth_agent(self, sex, mother_agent, father_agent):
        """ Creates a single new agent and sets it up in the network
            New agents inherit each gene at random from either parent
            They also inherit some of their mother's social connections
        """

        # Add new agent to the population
        new_agent = ap.AgentDList(self, 1, AnimalAgent)
        new_agent[0].sex = sex
        new_agent[0].age = 0
        new_agent[0].get_genome_from_parents(mother_agent.genome, father_agent.genome)
        new_agent[0].mutate_genome()
        self.pop += new_agent
        # Ensure the agent can 'see' the network
        self.pop.network = self.network
        # Set up the agent on a new node on the network
        self.network.add_agents(new_agent)
        # Give the agent a connection to its mother
        agent_node = self.network.positions[new_agent[0]]
        mother_node = self.network.positions[mother_agent]
        self.network.graph.add_edge(agent_node, mother_node)
        # Give it D random new associates (keeping the mean degree d on ave)
        D = np.random.binomial(self.p.N-2, (self.p.d-1)/(self.p.N-2))
        # Individuals connected to the mother are w times more likely to be selected
        mother_neighbors = list(model.network.neighbors(model.pop[0]))
        mother_neighbor_nodes = [model.network.positions[node] for node in mother_neighbors]
        connected_to_mother = [model.network.positions[agt] in mother_neighbor_nodes for agt in model.pop]
        probs = [self.p.w if connected else 1 for connected in connected_to_mother]
        probs = [ p/sum(probs) for p in probs]
        for i in range(D):
            rand_node = np.random.choice(list(self.network.graph.nodes),p=probs)
            # Check that associate isn't self or already connected
            while (rand_node==agent_node) or (self.network.graph.has_edge(agent_node, rand_node)):
                rand_node = np.random.choice(list(self.network.graph.nodes),p=probs)
            self.network.graph.add_edge(agent_node, rand_node)

    def roulette_wheel_select(self, candidate_agents):
        sum_fitnesses = sum(candidate_agents.fitness)
        selection_probs = [c.fitness/sum_fitnesses for c in candidate_agents]
        return candidate_agents[np.random.choice(len(candidate_agents), p=selection_probs)]

    def select_agent_to_kill(self):
        """ Returns which agent to kill.
        Works by selecting agents to survive and seeing who is left
        """
        self.pop.alive = False
        # N-1 agents to survive
        for _ in range(self.p.N-1):
            # Sum fitnesses (excluding agebnts already selected to survive)
            candidate_agents = self.pop.select(self.pop.alive == False)
            assert len(candidate_agents) > 0
            selected_agent = self.roulette_wheel_select(candidate_agents)
            selected_agent.alive = True
        
        return( self.pop.select(self.pop.alive == False) )

    def update(self):
        """ Called after setup and every step """
        
        # Record stats
        # self.pop.record("opinion")

    def step(self):
        """ Model events per timestep """

        # No need to shuffle order for this model because fitness updates are synchronous
        # self.pop.shuffle().opinion_influence()
        self.pop.social_interaction()
        self.pop.calculate_fitness()
        agent_to_kill = self.select_agent_to_kill()
        sex = agent_to_kill[0].sex
        self.kill_agent(agent_to_kill[0])
        mother_agent = self.pop.select(self.pop.sex == 'f')[0]
        father_agent = self.pop.select(self.pop.sex == 'm')[0]
        self.birth_agent(sex, mother_agent, father_agent)
        self.pop.increase_age()

    def end(self):
        """ Termination conditions """
        self.pop.social_interaction()
        self.pop.calculate_fitness()
        self.pop.increase_age()
        return

In [197]:
parameters = {
    'N': 100,        # population size
    'steps': 1000,   # Generations
    'm': 68,         # Age of maturity
    'w': 28,         # times more likely offspring select mothers' social partner over others
    'd': 19.42,      # Mean degree of the social network
    'mut_prob': 0.1, # Mutation probability
    'benefit': 1,
    'cost': 0.1,
    'fitness_baseline': 0.2
}

In [198]:
# Single run of the model (experiments later)
model = MainModel(parameters)
results = model.run()

Completed: 1000 steps
Run time: 0:00:10.930379
Simulation finished


In [199]:
# Males
sex = 0 # male
immature_cooperators_m = [g[sex,0] for g in model.pop.genome]
adult_cooperators_m = [g[sex,1] for g in model.pop.genome]

In [200]:
immature_cooperators_m.count('c')

57

In [201]:
adult_cooperators_m.count('c')

90

In [202]:
# Females
sex = 1 # female
immature_cooperators_f = [g[sex,0] for g in model.pop.genome]
adult_cooperators_f = [g[sex,1] for g in model.pop.genome]

In [203]:
immature_cooperators_f.count('c')

90

In [204]:
adult_cooperators_f.count('c')

7

In [205]:
import statistics
statistics.median(model.pop.age)

74.5

In [176]:
model.pop.fitness # Same individual is being selected to die each time

[0.8407407407407407, 0.6263157894736843, 0.6263157894736843, 0.45000000000000007, 0.9647058823529411, 0.5166666666666667, 0.7060606060606062, 0.5545454545454546, 0.7, 0.7882352941176471, 0.6545454545454545, 0.8500000000000001, 0.5444444444444444, 0.6789473684210526, 0.7, 0.825, 0.725, 0.9272727272727272, 0.9142857142857144, 0.5705882352941177, 0.7384615384615385, 0.6555555555555556, 0.7000000000000001, 1.2, 0.6263157894736843, 0.6, 0.6263157894736843, 0.8500000000000001, 0.8, 0.7833333333333334, 0.7238095238095239, 0.6294117647058824, 0.8500000000000001, 0.8272727272727273, 0.690909090909091, 0.7333333333333333, 0.6555555555555556, 0.9142857142857144, 0.7714285714285714, 0.6416666666666667, 0.7875, 0.9857142857142858, 0.7470588235294119, 0.5705882352941177, 0.831578947368421, 0.55, 0.7882352941176471, 0.6238095238095239, 0.5210526315789473, 0.7294117647058824, 0.690909090909091, 0.6652173913043479, 0.6652173913043479, 0.8058823529411766, 0.8400000000000001, 0.7500000000000001, 0.736363

In [206]:
# Get all logged data for agents
# results = results.variables.AnimalAgent
# Convert the resulting Pandas series to a dataFrame
# results = results.genome.to_frame().reset_index()

In [ ]:
# Stats to come